In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

from transformers import TapasForSequenceClassification, TapasTokenizer

import numpy as np 
import pandas as pd


In [95]:
# https://www.kaggle.com/datasets/rohitsahoo/employeeㄴ
train = pd.read_csv('/Users/ywkim/Downloads/archive/train.csv', encoding='ascii')
test = pd.read_csv('/Users/ywkim/Downloads/archive/test.csv', encoding='utf-8')

In [93]:
from sklearn.model_selection import train_test_split
training, val = train_test_split(train, test_size = .2, random_state = 42, stratify = train['Attrition'])

In [58]:
str_cols = train.select_dtypes(object).columns.tolist()

In [53]:
cat_to_str = {
    'Education':dict(zip(range(1,6), ['Below College',
'College',
'Bachelor',
'Master',
'Doctor'])),


'EnvironmentSatisfaction':dict(zip(range(1,5), [
    'Low',
'Medium',
'High',
'Very High'
])),


'JobInvolvement':dict(zip(range(1,5), [
    'Low',
'Medium',
'High',
'Very High'
])),

'JobSatisfaction':dict(zip(range(1,5), [
'Low',
'Medium',
'High',
'Very High'
])),

'PerformanceRating':dict(zip(range(1,5), [
'Low',
'Good',
'Excellent',
'Outstanding'
])),

'RelationshipSatisfaction':dict(zip(range(1,5), [
'Low',
'Medium',
'High',
'Very High'
])),


'WorkLifeBalance':dict(zip(range(1,5), [
'Bad',
'Good',
'Better',
'Best'
]))

}

In [62]:
for col in cat_to_str.keys():
    training[col] = training[col].apply(lambda x: cat_to_str[col][x]).copy()
    val[col] = val[col].apply(lambda x: cat_to_str[col][x]).copy()

In [50]:
import re
re.sub(r'[\(\)\{\}\']', '', str(dict(train.loc[0])))

'Age: 41, Attrition: 1, BusinessTravel: Travel_Rarely, DailyRate: 1102, Department: Sales, DistanceFromHome: 1, Education: 2, EducationField: Life Sciences, EmployeeCount: 1, EmployeeNumber: 1, EnvironmentSatisfaction: 2, Gender: Female, HourlyRate: 94, JobInvolvement: 3, JobLevel: 2, JobRole: Sales Executive, JobSatisfaction: 4, MaritalStatus: Single, MonthlyIncome: 5993, MonthlyRate: 19479, NumCompaniesWorked: 8, Over18: Y, OverTime: Yes, PercentSalaryHike: 11, PerformanceRating: 3, RelationshipSatisfaction: 1, StandardHours: 80, StockOptionLevel: 0, TotalWorkingYears: 8, TrainingTimesLastYear: 0, WorkLifeBalance: 1, YearsAtCompany: 6, YearsInCurrentRole: 4, YearsSinceLastPromotion: 0, YearsWithCurrManager: 5'

In [65]:
training_text = []
X_training = training.drop('Attrition', axis=1).copy()

for idx, row in X_training.iterrows():
    training_text.append(re.sub(r'[\(\)\{\}\']', '', str(dict(row))))


In [74]:
training_text[0:1]

['Age: 34, BusinessTravel: Travel_Rarely, DailyRate: 507, Department: Sales, DistanceFromHome: 15, Education: College, EducationField: Medical, EmployeeCount: 1, EmployeeNumber: 1356, EnvironmentSatisfaction: High, Gender: Female, HourlyRate: 66, JobInvolvement: High, JobLevel: 2, JobRole: Sales Executive, JobSatisfaction: Low, MaritalStatus: Single, MonthlyIncome: 6125, MonthlyRate: 23553, NumCompaniesWorked: 1, Over18: Y, OverTime: No, PercentSalaryHike: 12, PerformanceRating: Excellent, RelationshipSatisfaction: Very High, StandardHours: 80, StockOptionLevel: 0, TotalWorkingYears: 10, TrainingTimesLastYear: 6, WorkLifeBalance: Best, YearsAtCompany: 10, YearsInCurrentRole: 8, YearsSinceLastPromotion: 9, YearsWithCurrManager: 6']

In [100]:
train.astype(object).dtypes

Age                         object
Attrition                   object
BusinessTravel              object
DailyRate                   object
Department                  object
DistanceFromHome            object
Education                   object
EducationField              object
EmployeeCount               object
EmployeeNumber              object
EnvironmentSatisfaction     object
Gender                      object
HourlyRate                  object
JobInvolvement              object
JobLevel                    object
JobRole                     object
JobSatisfaction             object
MaritalStatus               object
MonthlyIncome               object
MonthlyRate                 object
NumCompaniesWorked          object
Over18                      object
OverTime                    object
PercentSalaryHike           object
PerformanceRating           object
RelationshipSatisfaction    object
StandardHours               object
StockOptionLevel            object
TotalWorkingYears   

In [ ]:
tokenizer(train.astype(str), training_text[0:1], max_length=512, truncation=True, )

In [51]:
train.columns[train.nunique()==1]

Index(['EmployeeCount', 'Over18', 'StandardHours'], dtype='object')

In [3]:
model = TapasForSequenceClassification.from_pretrained("google/tapas-base-finetuned-tabfact")

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

In [5]:
tokenizer = TapasTokenizer.from_pretrained("google/tapas-base-finetuned-tabfact")
# model = TapasForSequenceClassification.from_pretrained("google/tapas-base-finetuned-tabfact")

In [83]:
training.iloc[:1, :2].astype(object)

,Age,Attrition
964,34,0


In [86]:
table.iloc[0]

Age          34
Attrition     0
Name: 964, dtype: object

In [90]:
data = {
    "Actors": ["Brad Pitt", "Leonardo Di Caprio", "George Clooney"],
    "Age": ["56", "45", "59"],
    "Number of movies": ["87", "53", "69"],
}
# table = training.iloc[:1, :2].astype(object).reset_index().copy()
table =  pd.DataFrame.from_dict(data)
queries = [
    "There is only one actor who is 45 years old",
]

inputs = tokenizer(table=table, queries=queries, padding="max_length", return_tensors="pt", max_length=10000)
# labels =

In [78]:
training

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
964,34,0,Travel_Rarely,507,Sales,15,College,Medical,1,1356,...,Very High,80,0,10,6,Best,10,8,9,6
396,43,0,Travel_Rarely,1473,Research & Development,8,Master,Other,1,526,...,Very High,80,0,8,3,Better,5,2,0,2
946,40,1,Travel_Rarely,299,Sales,25,Master,Marketing,1,1318,...,High,80,0,9,2,Better,5,4,1,0
974,27,0,Travel_Frequently,793,Sales,2,Below College,Life Sciences,1,1371,...,Medium,80,0,8,3,Better,6,2,0,0
252,29,0,Travel_Rarely,665,Research & Development,15,Bachelor,Life Sciences,1,346,...,Low,80,0,6,1,Better,6,5,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,34,0,Travel_Rarely,629,Research & Development,27,College,Medical,1,247,...,Very High,80,0,9,3,Better,3,2,1,2
927,36,0,Travel_Rarely,429,Research & Development,2,Master,Life Sciences,1,1294,...,Very High,80,0,18,2,Better,16,14,5,12
73,32,0,Travel_Rarely,548,Research & Development,1,Bachelor,Life Sciences,1,96,...,Medium,80,2,10,3,Better,10,4,0,9
784,38,0,Travel_Rarely,330,Research & Development,17,Below College,Life Sciences,1,1088,...,Low,80,1,20,4,Good,19,9,1,9


In [76]:
table

,Actors,Age,Number of movies
0,Brad Pitt,56,87
1,Leonardo Di Caprio,45,53
2,George Clooney,59,69


In [23]:
inputs['input_ids'].shape, inputs['token_type_ids'].shape

(torch.Size([3, 512]), torch.Size([3, 512, 7]))

In [28]:
train.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

In [19]:
table

,Actors,Age,Number of movies
0,Brad Pitt,56,87
1,Leonardo Di Caprio,45,53
2,George Clooney,59,69


In [20]:
inputs['attention_mask'].shape

torch.Size([1, 512])

In [24]:
inputs['input_ids'][0, :30]

tensor([  101,  2045,  2003,  2069,  2028,  3364,  2040,  2003,  3429,  2086,
         2214,   102,  5889,  2287,  2193,  1997,  5691,  8226, 15091,  5179,
         6584, 14720,  4487,  6178,  9488,  3429,  5187,  2577, 18856,  7828])

In [25]:
inputs['input_ids'][1, :30]

tensor([  101,  2045,  2003,  2069,  2028,  3364,  2040,  2003,  3429,  2086,
         2214,   102,  5889,  2287,  2193,  1997,  5691,  8226, 15091,  5179,
         6584, 14720,  4487,  6178,  9488,  3429,  5187,  2577, 18856,  7828])